In [1]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

import re
import random
from multiprocessing import Pool
from tqdm.notebook import tqdm
import Levenshtein
from itertools import product
from collections import Counter, defaultdict
from functools import reduce
from spyt import spark_session
from nltk.util import everygrams
from sklearn.feature_extraction.text import CountVectorizer
import logging.config
import spyt
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import col, lit
from pyspark.sql.window import Window
from clan_tools.utils.spark import SPARK_CONF_LARGE
from clan_tools.data_adapters.YTAdapter import YTAdapter

warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 250)

In [2]:
from clan_tools.secrets.Vault import Vault
Vault().get_secrets('sec-01f47c9hp0b05jgy1r35rvx5n6')
yt_adapter = YTAdapter()
yt = yt_adapter.yt

spark = spyt.connect(spark_conf_args=SPARK_CONF_LARGE)
spyt.info(spark)

2021-12-22 22:39:01,774 - WARNING - spyt.conf - Your SPYT library has version 1.3.5 which is older than your cluster version 3.0.1-1.23.1+yandex. Some new features may not work as expected. Please update your cluster with spark-launch-yt utility
2021-12-22 22:39:16,880 - INFO - spyt.client - SPYT Cluster version: 3.0.1-1.23.1+yandex
2021-12-22 22:39:16,882 - INFO - spyt.client - SPYT library version: 1.3.5


### Initialize data

In [3]:
all_names = '//statbox/business-dwh/res/2021-12-06/beta/sources/spark'
mal_list = '//home/cloud_analytics/kulaga/acc_sales_ba_cube'

In [4]:
def safe_concat(spark_col1, spark_col2):
    return (
        F.when(col(spark_col1).isNull(), F.lower(col(spark_col2))).otherwise(
            F.when(col(spark_col2).isNull(), F.lower(col(spark_col1))).otherwise(
                F.concat(F.lower(col(spark_col1)), lit(' || '), F.lower(col(spark_col2)))
            )
        )
    )
           
mal_spdf = (
    spark.read.yt(mal_list)
    .filter(~col('acc_name').isNull())
    .filter(~col('inn').isNull())
    .filter(col('business_segment')=='Enterprise')
    .groupby('acc_name')
    .agg((F.collect_list('inn')[0]).alias('inn'))
)

spdf = (
    spark.read.yt(all_names)
    .filter(~col('inn').isNull())
    .filter(col('is_firm')==True)
    .filter(~col('name').isNull())
    .groupby('inn')
    .agg(F.concat_ws(" || ", F.collect_list('name')).alias('name'))
    .join(mal_spdf, on="inn", how='left')
    .select(
        safe_concat('name', 'acc_name').alias('name'),
        F.lower(col('name')).alias('spark_name'),
        F.lower(col('acc_name')).alias('mal_name'),
        col('inn')
    )
    .groupby('name')
    .agg(
        F.concat_ws(" || ", F.collect_list('spark_name')).alias('spark_name'),
        F.concat_ws(" || ", F.collect_list('mal_name')).alias('mal_name'),
        F.concat_ws(", ", F.collect_list(col('inn').astype('string'))).alias('inn'),
    )
    .distinct()
)

### Generation of keys

In [5]:
def find_key_words(company_name: str):
    name = (
        company_name
        .replace(' | ', ' || ')
        .replace(' / ', ' || ')
        .replace('\\', ' || ')
        .replace('в прошлом', ' || ')
    )
    
    name = re.sub(r"[^\w\.\-&\+\'@\|]+", " ", name).strip()
    all_keys = []
    for subname in name.split('||'):
        subname = subname.strip()
        keys = [' '.join(words) for words in everygrams(subname.split(" ")) if ' '.join(words)!='']
        all_keys.extend(keys)

    return list(set(all_keys))

def list_of_all_words(company_name: str):
    name = (
        company_name
        .replace(' | ', ' || ')
        .replace(' / ', ' || ')
        .replace('\\', ' || ')
        .replace('в прошлом', ' || ')
    )
    
    name = re.sub(r"[^\w\.\-&\+\'@\|]+", " ", name).strip()
    return name.split(' ')

def K_affinity_levenstein(word1, word2, lower_limit=None):
    max_len = max(len(word1), len(word2))
    lev_dist = Levenshtein.distance(word1, word2)
    k_aff_lev = 1.0 - lev_dist/max_len
    if lower_limit is None:
        return k_aff_lev
    return k_aff_lev if k_aff_lev >= lower_limit else 0.0

def K_sorensen_levenstein(token_list_a, token_list_b, lower_limit=0.8):
    a, b = len(token_list_a), len(token_list_b)
    c = sum([K_affinity_levenstein(word_a, word_b, lower_limit) for word_a, word_b in product(token_list_a, token_list_b)])
    return 2 * c / (a + b)

class NLP_decomposition:
    word_pattern = re.compile(r"[\w\.\-&\+\'@\|]+")
    nonword_pattern = re.compile(r"[^\w\.\-&\+\'@\|]+")
    def __init__(self, max_df=5, max_df_key=50):
        self.max_df = max_df
        self.max_df_key = max_df_key

        sl = pd.read_csv('stoplist.csv')
        self.stoplist_ = sl['stopwords'].tolist()
        self.stoplist_.append('')
        self.repeats_ = []
        self.all_banned = None
        self.used_keys = defaultdict(int)

    def _meets_reqs(self, phrase):
        phrase = re.sub(r"^[\d\.\-&\+\'@\|]+$", "", phrase)
        if (phrase in self.all_banned) or (len(phrase)<=2):
            return False
        if (self.used_keys[phrase]>self.max_df_key):
            return False
        subwords = re.findall(self.word_pattern, phrase)
        if subwords[0] in self.all_banned:
            return False
        if subwords[-1] in self.all_banned:
            return False
        self.used_keys[phrase] += 1
        return True

    @staticmethod
    def transliterate(string):
        lower_case_letters = {
            u'а': u'a', u'б': u'b', u'в': u'v', u'г': u'g', u'д': u'd', u'е': u'e', u'ё': u'e',
            u'ж': u'zh', u'з': u'z', u'и': u'i', u'й': u'y', u'к': u'k', u'л': u'l', u'м': u'm',
            u'н': u'n', u'о': u'o', u'п': u'p', u'р': u'r', u'с': u's', u'т': u't', u'у': u'u',
            u'ф': u'f', u'х': u'h', u'ц': u'ts', u'ч': u'ch', u'ш': u'sh', u'щ': u'sch', u'ъ': u'',
            u'ы': u'y', u'ь': u'', u'э': u'e', u'ю': u'yu', u'я': u'ya',
        }

        for cyrillic_string, latin_string in lower_case_letters.items():
            string = string.replace(cyrillic_string, latin_string)
        return string

    def transform(self, df: pd.DataFrame):
        def process_delimiters(word):
            return (
                word
                .replace(' | ', ' || ')
                .replace(' / ', ' || ')
                .replace('\\', ' || ')
                .replace('в прошлом', ' || ')
            )

        # all filtered names
        norm_delimiters = [process_delimiters(name) for name in df['name']]
        data_corpus = [re.sub(self.nonword_pattern, " ", name).strip() for name in norm_delimiters]
        
        # find repeats
        print('Search of repeats...')
        all_keys_total = []
        for name in tqdm(data_corpus, total=len(data_corpus)):
            for subname in name.split('||'):
                subname = subname.strip()
                for lang_name in [subname, self.transliterate(subname)]:
                    tokens = [' '.join(words) for words in everygrams(lang_name.split(" "))]
                    all_keys_total.extend(tokens)
        all_keys_total = pd.Series(Counter(all_keys_total)).reset_index()
        all_keys_total.columns = ['keyword', 'cnt']
        all_keys_total['len'] = all_keys_total['keyword'].apply(lambda x: len(x.strip().split(" ")))
        new_stopwords = all_keys_total[(all_keys_total['len']==1) & (all_keys_total['cnt']>100)]['keyword']
        self.stoplist_.extend(new_stopwords)
        new_reps = all_keys_total[(all_keys_total['len']>1) & (all_keys_total['cnt']>self.max_df)]['keyword']
        self.repeats_ = new_reps.tolist()
        self.all_banned = set(self.stoplist_) | set(self.repeats_)

        # generate name-keys
        print('Generate keys...')
        all_keys = []
        for name in tqdm(data_corpus, total=len(data_corpus)):
            res_keys = []
            for subname in name.split('||'):
                subname = subname.strip()
                for lang_name in [subname, self.transliterate(subname)]:
                    keys = [' '.join(words) for words in everygrams(lang_name.split(" "))]
                    keys = [key for key in keys if self._meets_reqs(key)]
                    res_keys.extend(keys)
            all_keys.append(list(set(res_keys)))

        df_res = df.copy()
        df_res['raw_keys'] = all_keys
        return df_res

In [6]:
def leave_only_N_names(name, N=5):
    new_name = ' || '.join(name.split(' || ')[:N])
    return new_name

dft = spdf.toPandas()
dft['len'] = dft['name'].apply(lambda x: len(find_key_words(x)))
dft['name'] = dft['name'].apply(lambda x: leave_only_N_names(x, N=5))
dft = dft[dft['len']>0].sort_values('len', ascending=False).reset_index(drop=True)
print('Num_rows:', dft.shape[0])
dft.head()

Num_rows: 1801858


,name,spark_name,mal_name,inn,len
0,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,1046
1,"активо шесть, зпиф недвижимости || альфа-капит...","активо шесть, зпиф недвижимости || альфа-капит...","ооо ук ""альфа-капитал""",7728142469,771
2,тинькофф-стратегия вечного портфеля в долларах...,тинькофф-стратегия вечного портфеля в долларах...,,7743304530,664
3,местная религиозная организация православный п...,местная религиозная организация православный п...,,2801200083,628
4,"подворье в честь святых мучениц веры, надежды,...","подворье в честь святых мучениц веры, надежды,...",,2801177050,626


In [7]:
decomposer = NLP_decomposition()
ttdf = dft
ttdf = decomposer.transform(ttdf)
ttdf

Search of repeats...


  0%|          | 0/1801858 [00:00<?, ?it/s]

Generate keys...


  0%|          | 0/1801858 [00:00<?, ?it/s]

,name,spark_name,mal_name,inn,len,raw_keys
0,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,1046,"[vtb -fond subordinirovannyy., -фонд субордини..."
1,"активо шесть, зпиф недвижимости || альфа-капит...","активо шесть, зпиф недвижимости || альфа-капит...","ооо ук ""альфа-капитал""",7728142469,771,"[активо, интерфин, interfin, alfa-kapital stre..."
2,тинькофф-стратегия вечного портфеля в долларах...,тинькофф-стратегия вечного портфеля в долларах...,,7743304530,664,"[рублях, тинькофф-стратегия вечного портфеля в..."
3,местная религиозная организация православный п...,местная религиозная организация православный п...,,2801200083,628,"[uchiteley i svyatiteley, svyatiteley vasiliya..."
4,"подворье в честь святых мучениц веры, надежды,...","подворье в честь святых мучениц веры, надежды,...",,2801177050,626,"[eparhialnogo zhenskogo, женского монастыря в ..."
...,...,...,...,...,...,...
1801853,апокроо,апокроо,,5506177097,1,"[apokroo, апокроо]"
1801854,кэфт,кэфт,,4346046736,1,"[кэфт, keft]"
1801855,аноасб,аноасб,,2367008520,1,"[аноасб, anoasb]"
1801856,басо,басо,,6732191435,1,"[baso, басо]"


In [8]:
tmp = ttdf[ttdf['raw_keys'].apply(len)>0].reset_index(drop=True)
res_df = []
for ind in tqdm(tmp.index, total=tmp.shape[0]):
    tmp_name = tmp.loc[ind, 'name']
    tmp_spark_name = tmp.loc[ind, 'spark_name']
    tmp_mal_name = tmp.loc[ind, 'mal_name']
    tmp_inn = tmp.loc[ind, 'inn']
    tmp_raw_keys = tmp.loc[ind, 'raw_keys']
    for tmp_key in tmp_raw_keys:
        res_df.append({
            'name': tmp_name,
            'spark_name': tmp_spark_name,
            'mal_name': tmp_mal_name,
            'inn': tmp_inn,
            'key': tmp_key,
            'spark_keys': tmp_raw_keys,
            'one_word_spark_keys': list_of_all_words(tmp_name)
        })

res_df = pd.DataFrame(res_df)
res_df['mal_name'] = res_df['mal_name'].fillna('').astype(str)
res_df['inn'] = res_df['inn'].astype(str)
print(res_df.shape[0])
res_df.head()

  0%|          | 0/1482798 [00:00<?, ?it/s]

4198711


,name,spark_name,mal_name,inn,key,spark_keys,one_word_spark_keys
0,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,vtb -fond subordinirovannyy.,"[vtb -fond subordinirovannyy., -фонд субордини...","[втб-фонд, сбалансированный, опиф, рыночных, ф..."
1,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,-фонд субординированный. рубль ипиф,"[vtb -fond subordinirovannyy., -фонд субордини...","[втб-фонд, сбалансированный, опиф, рыночных, ф..."
2,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,rubl ipif,"[vtb -fond subordinirovannyy., -фонд субордини...","[втб-фонд, сбалансированный, опиф, рыночных, ф..."
3,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,rtk-razvitie,"[vtb -fond subordinirovannyy., -фонд субордини...","[втб-фонд, сбалансированный, опиф, рыночных, ф..."
4,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,втб резерв зпи,"[vtb -fond subordinirovannyy., -фонд субордини...","[втб-фонд, сбалансированный, опиф, рыночных, ф..."


### Match all comapnies from MAL list

In [9]:
mal_spdf_all = (
    spark.read.yt(mal_list)
    .filter(~col('acc_name').isNull())
    .filter(col('business_segment')=='Enterprise')
    .select(F.lower('acc_name').alias('acc_name'))
    .distinct()
)

df_mal_all = mal_spdf_all.toPandas()
found_companies = res_df[['mal_name']][res_df['mal_name']!=''].drop_duplicates().reset_index(drop=True)
df_mal_all_n_found = df_mal_all.merge(found_companies, left_on='acc_name', right_on='mal_name', how='left')
df_mal_not_found = df_mal_all_n_found[df_mal_all_n_found['mal_name'].isna()][['acc_name']].reset_index(drop=True)

In [10]:
df_mal_nf_with_keys = df_mal_not_found.copy()
df_mal_nf_with_keys['keys'] = df_mal_nf_with_keys['acc_name'].apply(find_key_words)
df_mal_nf_with_keys.head()

,acc_name,keys
0,семейный доктор,"[семейный доктор, доктор, семейный]"
1,"ао ""ск донстрой""","[ао ск донстрой, ск, донстрой, ао ск, ск донст..."
2,зао европа плюс \\ европа плюс,"[европа, зао европа, европа плюс, плюс, зао ев..."
3,ооо предприятие авторадио \\ авторадио,"[ооо предприятие, ооо, предприятие авторадио, ..."
4,зао тв дарьял \\ че,"[че, тв дарьял, зао тв дарьял, зао тв, дарьял,..."


In [11]:
df_mal_nf_with_keys_long = []
for ind in tqdm(df_mal_nf_with_keys.index, total=df_mal_nf_with_keys.shape[0]):
    for key in df_mal_nf_with_keys.loc[ind, 'keys']:
        df_mal_nf_with_keys_long.append({
            'acc_name': df_mal_nf_with_keys.loc[ind, 'acc_name'],
            'key': key,
            'mal_keys': df_mal_nf_with_keys.loc[ind, 'keys'],
            'one_word_mal_keys': list_of_all_words(df_mal_nf_with_keys.loc[ind, 'acc_name'])
        })

df_mal_nf_with_keys_long = pd.DataFrame(df_mal_nf_with_keys_long)
df_mal_nf_with_keys_long.head()

  0%|          | 0/522 [00:00<?, ?it/s]

,acc_name,key,mal_keys,one_word_mal_keys
0,семейный доктор,семейный доктор,"[семейный доктор, доктор, семейный]","[семейный, доктор]"
1,семейный доктор,доктор,"[семейный доктор, доктор, семейный]","[семейный, доктор]"
2,семейный доктор,семейный,"[семейный доктор, доктор, семейный]","[семейный, доктор]"
3,"ао ""ск донстрой""",ао ск донстрой,"[ао ск донстрой, ск, донстрой, ао ск, ск донст...","[ао, ск, донстрой]"
4,"ао ""ск донстрой""",ск,"[ао ск донстрой, ск, донстрой, ао ск, ск донст...","[ао, ск, донстрой]"


In [12]:
def merge_max(df_mal, df_spark, strength=0.5):
    t = df_mal.merge(df_spark, on='key', how='inner')
    t = t[t['mal_name']==''].reset_index(drop=True)
    for ind in tqdm(t.index.tolist(), total=t.shape[0]):
        t.loc[ind, 'score_1'] = K_sorensen_levenstein(t.loc[ind, 'mal_keys'], t.loc[ind, 'spark_keys'])
        t.loc[ind, 'score_2'] = K_sorensen_levenstein(t.loc[ind, 'one_word_mal_keys'],
                                                      t.loc[ind, 'one_word_spark_keys'])

    res = []
    for acc_name in tqdm(t['acc_name'].unique(), total=len(t['acc_name'].unique())):
        tt = t[t['acc_name']==acc_name]
        score_1_max = tt['score_1'].max()
        score_2_max = tt['score_2'].max()
        tt = tt[(tt['score_1']==score_1_max) & (tt['score_2']==score_2_max)]
        if (tt.shape[0] == 1) and (score_2_max >= strength):
            res.append(tt)

    return pd.concat(res, axis=0, ignore_index=True)

In [13]:
additional_mal = merge_max(df_mal_nf_with_keys_long, res_df)

  0%|          | 0/4265 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

In [14]:
additional_mal.head()

,acc_name,key,mal_keys,one_word_mal_keys,name,spark_name,mal_name,inn,spark_keys,one_word_spark_keys,score_1,score_2
0,"ао ""ск донстрой""",донстрой,"[ао ск донстрой, ск, донстрой, ао ск, ск донст...","[ао, ск, донстрой]","донстрой, ао","донстрой, ао",,6154012871,"[donstroy, донстрой]","[донстрой, ао]",0.250000,0.800000
1,ооо предприятие авторадио \\ авторадио,авторадио,"[ооо предприятие, ооо, предприятие авторадио, ...","[ооо, предприятие, авторадио, ||, ||, авторадио]","авторадио, ооо","авторадио, ооо || авторадио, ооо || авторадио,...",,"7024019677, 2801100949, 1841070047","[avtoradio, авторадио]","[авторадио, ооо]",0.250000,0.750000
2,ооо юмор фм \\ юмор fm,юмор,"[fm, ооо, юмор фм, юмор fm, фм, ооо юмор фм, ю...","[ооо, юмор, фм, ||, ||, юмор, fm]","пенза юмор, ооо","пенза юмор, ооо",,5837052451,"[юмор, yumor]","[пенза, юмор, ооо]",0.200000,0.600000
3,asteriosoft астерио,астерио,"[asteriosoft, астерио, asteriosoft астерио]","[asteriosoft, астерио]","астерио, ооо","астерио, ооо || астерио, ооо || астерио, ооо",,"7751164580, 7816676029, 7736334083","[asterio, астерио]","[астерио, ооо]",0.400000,0.500000
4,ооо «вэлфер» \\ sociate,вэлфер,"[sociate, ооо, вэлфер, ооо вэлфер]","[ооо, вэлфер, ||, ||, sociate]","вэлфер, ооо","вэлфер, ооо",,2311151077,"[velfer, вэлфер]","[вэлфер, ооо]",0.333333,0.571429


In [15]:
res_df2 = res_df.merge(additional_mal[['name', 'spark_name', 'inn', 'acc_name']],
                        on=['name', 'spark_name', 'inn'], how='left')
res_df2['mal_name'] = res_df2['mal_name'] + res_df2['acc_name'].fillna('')
yt_columns = ['name', 'spark_name', 'mal_name', 'inn', 'key', 'spark_keys', 'one_word_spark_keys']
res_df2 = res_df2[yt_columns]
res_df2.head(3)

,name,spark_name,mal_name,inn,key,spark_keys,one_word_spark_keys
0,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,vtb -fond subordinirovannyy.,"[vtb -fond subordinirovannyy., -фонд субордини...","[втб-фонд, сбалансированный, опиф, рыночных, ф..."
1,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,-фонд субординированный. рубль ипиф,"[vtb -fond subordinirovannyy., -фонд субордини...","[втб-фонд, сбалансированный, опиф, рыночных, ф..."
2,"втб-фонд сбалансированный, опиф рыночных финан...","втб-фонд сбалансированный, опиф рыночных финан...",,7701140866,rubl ipif,"[vtb -fond subordinirovannyy., -фонд субордини...","[втб-фонд, сбалансированный, опиф, рыночных, ф..."


### Saving data

In [16]:
yt_schema = [
        {"name": "name", "type": "string"},
        {"name": "spark_name", "type": "string"},
        {"name": "mal_name", "type": "string"},
        {"name": "inn", "type": "string"},
        {"name": "key", "type": "string"},
        {"name": "spark_keys", "type": "any"},
        {"name": "one_word_spark_keys", "type": "any"},
    ]

res_path = '//home/cloud_analytics/ml/mql_marketing/result/company_names_mapping/spark_company_keys'
# yt_adapter.yt.remove(res_path)
yt_adapter.save_result(result_path=res_path, schema=yt_schema, df=res_df2, append=False)
yt_adapter.optimize_chunk_number(res_path)

2021-12-22 22:57:05,638 - INFO - clan_tools.data_adapters.YTAdapter - Saving data to //home/cloud_analytics/ml/mql_marketing/result/company_names_mapping/spark_company_keys
2021-12-22 23:34:14,777 - INFO - clan_tools.data_adapters.YTAdapter - Results are saved
2021-12-22 23:34:14,778 - INFO - clan_tools.utils.timing - func: save_result took: 2229.141 sec, 2045.742 sec CPU time.
2021-12-22 23:34:14,779 - INFO - clan_tools.data_adapters.YTAdapter - Optimizing table in path //home/cloud_analytics/ml/mql_marketing/result/company_names_mapping/spark_company_keys...
2021-12-22 23:34:16,555	INFO	Operation started: http://hahn.yt.yandex.net/?page=operation&mode=detail&id=da5bde24-b8d6da3c-3fe03e8-8a6dedc4&tab=details
2021-12-22 23:34:16,630 ( 0 min)	operation da5bde24-b8d6da3c-3fe03e8-8a6dedc4 starting
2021-12-22 23:34:17,725 ( 0 min)	operation da5bde24-b8d6da3c-3fe03e8-8a6dedc4 initializing
2021-12-22 23:34:18,964 ( 0 min)	Unrecognized spec: {'legacy_controller_fraction': 0}
2021-12-22 23:34: